In [2]:
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
from sklearn.model_selection import train_test_split

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [6]:
# Ganti path sesuai lokasi file Anda
df = pd.read_csv('../data/Bot_IoT_processedV2.csv')

# Tampilkan info dasar
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26265 entries, 0 to 26264
Data columns (total 41 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   pkts             26265 non-null  float64
 1   bytes            26265 non-null  float64
 2   dur              26265 non-null  float64
 3   mean             26265 non-null  float64
 4   stddev           26265 non-null  float64
 5   sum              26265 non-null  float64
 6   min              26265 non-null  float64
 7   max              26265 non-null  float64
 8   spkts            26265 non-null  float64
 9   dpkts            26265 non-null  float64
 10  sbytes           26265 non-null  float64
 11  dbytes           26265 non-null  float64
 12  rate             26265 non-null  float64
 13  srate            26265 non-null  float64
 14  drate            26265 non-null  float64
 15  sport            26265 non-null  int64  
 16  dport            26265 non-null  int64  
 17  proto_icmp  

,pkts,bytes,dur,mean,stddev,sum,min,max,spkts,dpkts,...,flgs_e F,flgs_e t,flgs_e &,flgs_e *,flgs_e d,flgs_e g,flgs_e r,flgs_e s,flgs_eU,attack
0,-0.077967,-0.071110,-0.111816,-0.183322,-0.106786,-0.105419,-0.169021,-0.191095,-0.080777,-0.051129,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
1,-0.077967,-0.071119,-0.135191,-0.183126,-0.106786,-0.105418,-0.168765,-0.190916,-0.081002,-0.050742,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
2,-0.077967,-0.071119,-0.135035,-0.125480,-0.106786,-0.105234,-0.093207,-0.138104,-0.081002,-0.050742,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
3,-0.077967,-0.071119,-0.135041,-0.127609,-0.106786,-0.105241,-0.095998,-0.140055,-0.081002,-0.050742,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
4,-0.077333,-0.071029,-0.135190,-0.182903,-0.106786,-0.105418,-0.168473,-0.190712,-0.080552,-0.049967,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
